In [1]:
import numpy as np;
import pandas as pd;
from sklearn.linear_model import LinearRegression;
from sklearn.metrics import mean_squared_error;
from sklearn.model_selection import cross_val_score;
from sklearn.preprocessing import PolynomialFeatures; 
import seaborn as sbn;
import datetime;
%matplotlib inline

In [2]:
org_train = pd.read_csv("train_kOBLwZA.csv");
org_test = pd.read_csv("test_t02dQwI.csv");
org_sam = pd.read_csv("SampleSubmission_TmnO39y.txt")

In [3]:
print(org_train.shape,org_test.shape,org_sam.shape)

(8523, 12) (5681, 11) (5681, 3)


In [4]:
org_data = pd.concat([org_train,org_test],axis="index",sort=False)
org_data.shape

(14204, 12)

In [5]:
org_data.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,11765.000000,14204.000000,14204.000000,14204.000000,8523.000000
mean,12.792854,0.065953,141.004977,1997.830681,2181.288914
std,4.652502,0.051459,62.086938,8.371664,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.710000,0.027036,94.012000,1987.000000,834.247400
50%,12.600000,0.054021,142.247000,1999.000000,1794.331000
75%,16.750000,0.094037,185.855600,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [6]:
org_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14204 entries, 0 to 5680
Data columns (total 12 columns):
Item_Identifier              14204 non-null object
Item_Weight                  11765 non-null float64
Item_Fat_Content             14204 non-null object
Item_Visibility              14204 non-null float64
Item_Type                    14204 non-null object
Item_MRP                     14204 non-null float64
Outlet_Identifier            14204 non-null object
Outlet_Establishment_Year    14204 non-null int64
Outlet_Size                  10188 non-null object
Outlet_Location_Type         14204 non-null object
Outlet_Type                  14204 non-null object
Item_Outlet_Sales            8523 non-null float64
dtypes: float64(4), int64(1), object(7)
memory usage: 1.4+ MB


In [7]:
org_data.duplicated().sum()

0

In [8]:
org_data.isna().sum()

Item_Identifier                 0
Item_Weight                  2439
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  4016
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales            5681
dtype: int64

In [9]:
org_data["Item_Identifier"].apply(lambda x : x[:3]).unique()

array(['FDA', 'DRC', 'FDN', 'FDX', 'NCD', 'FDP', 'FDO', 'FDH', 'FDU',
       'FDY', 'FDS', 'FDF', 'NCB', 'DRI', 'FDW', 'FDC', 'FDR', 'FDV',
       'DRJ', 'FDE', 'NCS', 'DRH', 'NCX', 'DRZ', 'FDB', 'FDK', 'FDL',
       'FDM', 'NCP', 'NCL', 'DRK', 'FDI', 'FDZ', 'NCI', 'FDJ', 'FDG',
       'NCZ', 'FDQ', 'FDD', 'DRG', 'NCR', 'FDT', 'DRB', 'DRE', 'DRA',
       'NCF', 'NCH', 'NCO', 'NCN', 'NCC', 'DRD', 'DRF', 'DRL', 'NCM',
       'NCU', 'DRY', 'NCW', 'DRM', 'NCT', 'NCQ', 'DRP', 'DRQ', 'NCK',
       'NCY', 'DRN', 'NCA', 'NCE', 'NCJ', 'NCV', 'NCG', 'DRO'],
      dtype=object)

In [10]:
org_data["Item_Identifier_type"] = org_data["Item_Identifier"].apply(lambda x : x[:3])

In [11]:
org_data["Item_Weight"].isna().sum()

2439

In [12]:
org_data["Item_Weight"].describe()

count    11765.000000
mean        12.792854
std          4.652502
min          4.555000
25%          8.710000
50%         12.600000
75%         16.750000
max         21.350000
Name: Item_Weight, dtype: float64

In [13]:
org_data["Item_Weight"] = org_data.groupby("Item_Identifier_type").Item_Weight.transform(lambda x: x.fillna(x.mean()))

In [14]:
org_data["Item_Fat_Content"].unique()

array(['Low Fat', 'Regular', 'low fat', 'LF', 'reg'], dtype=object)

In [15]:
org_data["Item_Fat_Content"].replace({"low fat" : "Low Fat" , "LF": "Low Fat", "reg" : "Regular"},inplace = True)

In [16]:
org_data["Item_Visibility"].isna().sum()

0

In [17]:
org_data["Item_Type"].isna().sum()

0

In [18]:
org_data["Item_Type"].unique()

array(['Dairy', 'Soft Drinks', 'Meat', 'Fruits and Vegetables',
       'Household', 'Baking Goods', 'Snack Foods', 'Frozen Foods',
       'Breakfast', 'Health and Hygiene', 'Hard Drinks', 'Canned',
       'Breads', 'Starchy Foods', 'Others', 'Seafood'], dtype=object)

In [19]:
org_data["Item_MRP"].isna().sum()

0

In [20]:
org_data["Item_MRP"].unique()

array([249.8092,  48.2692, 141.618 , ...,  72.8696,  82.125 , 188.053 ])

In [21]:
org_data["Outlet_Identifier"].isna().sum()

0

In [22]:
org_data["Outlet_Identifier"].unique()

array(['OUT049', 'OUT018', 'OUT010', 'OUT013', 'OUT027', 'OUT045',
       'OUT017', 'OUT046', 'OUT035', 'OUT019'], dtype=object)

In [23]:
now = datetime.datetime.now();
org_data["Outlet_Establishment_Year"] = now.year - org_data["Outlet_Establishment_Year"]

In [24]:
org_data["Outlet_Size"]

0       Medium
1       Medium
2       Medium
3          NaN
4         High
5       Medium
6         High
7       Medium
8          NaN
9          NaN
10      Medium
11       Small
12      Medium
13       Small
14        High
15       Small
16      Medium
17      Medium
18      Medium
19       Small
20        High
21      Medium
22       Small
23       Small
24       Small
25         NaN
26       Small
27        High
28         NaN
29       Small
         ...  
5651      High
5652       NaN
5653     Small
5654      High
5655    Medium
5656    Medium
5657     Small
5658      High
5659    Medium
5660      High
5661    Medium
5662       NaN
5663    Medium
5664    Medium
5665    Medium
5666     Small
5667     Small
5668       NaN
5669    Medium
5670       NaN
5671    Medium
5672    Medium
5673    Medium
5674     Small
5675    Medium
5676     Small
5677    Medium
5678       NaN
5679       NaN
5680       NaN
Name: Outlet_Size, Length: 14204, dtype: object

In [25]:
org_data["Outlet_Size"] = org_data.groupby("Outlet_Location_Type").Outlet_Size.transform(lambda x: x.fillna(x.mode()[0]))

In [26]:
org_data["Outlet_Type"]

0       Supermarket Type1
1       Supermarket Type2
2       Supermarket Type1
3           Grocery Store
4       Supermarket Type1
5       Supermarket Type2
6       Supermarket Type1
7       Supermarket Type3
8       Supermarket Type1
9       Supermarket Type1
10      Supermarket Type1
11      Supermarket Type1
12      Supermarket Type1
13      Supermarket Type1
14      Supermarket Type1
15      Supermarket Type1
16      Supermarket Type2
17      Supermarket Type1
18      Supermarket Type3
19      Supermarket Type1
20      Supermarket Type1
21      Supermarket Type3
22      Supermarket Type1
23          Grocery Store
24      Supermarket Type1
25      Supermarket Type1
26      Supermarket Type1
27      Supermarket Type1
28          Grocery Store
29          Grocery Store
              ...        
5651    Supermarket Type1
5652    Supermarket Type1
5653    Supermarket Type1
5654    Supermarket Type1
5655    Supermarket Type3
5656    Supermarket Type3
5657    Supermarket Type1
5658    Supe

In [27]:
org_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14204 entries, 0 to 5680
Data columns (total 13 columns):
Item_Identifier              14204 non-null object
Item_Weight                  14204 non-null float64
Item_Fat_Content             14204 non-null object
Item_Visibility              14204 non-null float64
Item_Type                    14204 non-null object
Item_MRP                     14204 non-null float64
Outlet_Identifier            14204 non-null object
Outlet_Establishment_Year    14204 non-null int64
Outlet_Size                  14204 non-null object
Outlet_Location_Type         14204 non-null object
Outlet_Type                  14204 non-null object
Item_Outlet_Sales            8523 non-null float64
Item_Identifier_type         14204 non-null object
dtypes: float64(4), int64(1), object(8)
memory usage: 1.5+ MB


In [28]:
cat_col = org_data.select_dtypes(exclude= np.number)
num_col = org_data.select_dtypes(include= np.number)

In [29]:
print("cat_col , num_col", cat_col, num_col)

cat_col , num_col      Item_Identifier Item_Fat_Content              Item_Type  \
0              FDA15          Low Fat                  Dairy   
1              DRC01          Regular            Soft Drinks   
2              FDN15          Low Fat                   Meat   
3              FDX07          Regular  Fruits and Vegetables   
4              NCD19          Low Fat              Household   
5              FDP36          Regular           Baking Goods   
6              FDO10          Regular            Snack Foods   
7              FDP10          Low Fat            Snack Foods   
8              FDH17          Regular           Frozen Foods   
9              FDU28          Regular           Frozen Foods   
10             FDY07          Low Fat  Fruits and Vegetables   
11             FDA03          Regular                  Dairy   
12             FDX32          Regular  Fruits and Vegetables   
13             FDS46          Regular            Snack Foods   
14             FDF32  

In [30]:
dum_data = pd.get_dummies(cat_col.drop(columns='Item_Identifier'))

In [31]:
mod_data = pd.concat([dum_data,num_col,org_data["Item_Identifier"]],axis="columns")
mod_data.shape

(14204, 115)

In [32]:
mod_data

,Item_Fat_Content_Low Fat,Item_Fat_Content_Regular,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,Item_Type_Frozen Foods,Item_Type_Fruits and Vegetables,Item_Type_Hard Drinks,...,Item_Identifier_type_NCW,Item_Identifier_type_NCX,Item_Identifier_type_NCY,Item_Identifier_type_NCZ,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales,Item_Identifier
0,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,9.300000,0.016047,249.8092,20,3735.1380,FDA15
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,5.920000,0.019278,48.2692,10,443.4228,DRC01
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,17.500000,0.016760,141.6180,20,2097.2700,FDN15
3,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,19.200000,0.000000,182.0950,21,732.3800,FDX07
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,8.930000,0.000000,53.8614,32,994.7052,NCD19
5,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,10.395000,0.000000,51.4008,10,556.6088,FDP36
6,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,13.650000,0.012741,57.6588,32,343.5528,FDO10
7,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,13.907747,0.127470,107.7622,34,4022.7636,FDP10
8,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,16.200000,0.016687,96.9726,17,1076.5986,FDH17
9,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,19.200000,0.094450,187.8214,12,4710.5350,FDU28


In [33]:
train = mod_data[mod_data["Item_Outlet_Sales"].notna()]
test = mod_data[mod_data["Item_Outlet_Sales"].isna()]
print(train.shape,test.shape)
train

(8523, 115) (5681, 115)


,Item_Fat_Content_Low Fat,Item_Fat_Content_Regular,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,Item_Type_Frozen Foods,Item_Type_Fruits and Vegetables,Item_Type_Hard Drinks,...,Item_Identifier_type_NCW,Item_Identifier_type_NCX,Item_Identifier_type_NCY,Item_Identifier_type_NCZ,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales,Item_Identifier
0,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,9.300000,0.016047,249.8092,20,3735.1380,FDA15
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,5.920000,0.019278,48.2692,10,443.4228,DRC01
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,17.500000,0.016760,141.6180,20,2097.2700,FDN15
3,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,19.200000,0.000000,182.0950,21,732.3800,FDX07
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,8.930000,0.000000,53.8614,32,994.7052,NCD19
5,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,10.395000,0.000000,51.4008,10,556.6088,FDP36
6,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,13.650000,0.012741,57.6588,32,343.5528,FDO10
7,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,13.907747,0.127470,107.7622,34,4022.7636,FDP10
8,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,16.200000,0.016687,96.9726,17,1076.5986,FDH17
9,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,19.200000,0.094450,187.8214,12,4710.5350,FDU28


In [34]:
x_train = train.drop(columns=["Item_Outlet_Sales", "Item_Identifier"])
x_train.shape

(8523, 113)

In [35]:
y_train = train["Item_Outlet_Sales"]
y_train.shape

(8523,)

In [36]:
lin = LinearRegression()

In [37]:
lin.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [38]:
y_train_pred = lin.predict(x_train);

In [39]:
x_test = test.drop(columns=["Item_Outlet_Sales", "Item_Identifier"])
y_test_predict = lin.predict(x_test)
y_test_predict

array([1776.77587891, 1434.70751953, 1857.23193359, ..., 1849.94726562,
       3533.01611328, 1312.30371094])

In [40]:
org_sam["Item_Outlet_Sales"] = y_test_predict

In [41]:
org_sam.to_csv("test_result.csv", index= False)

# Doing Linear Regression Sample

In [42]:
def model_test(model,x,y,test,fileName):
    model.fit(x,y);
    y_pred = model.predict(x);
    #crs_mse = np.mean(np.abs(cross_val_score(model,x,y,cv=20,scoring="neg_mean_squared_error")))
    mse = mean_squared_error(y,y_pred)
    #print('Train Cross Validation MSE', crs_mse)
    #print('Train Cross Validation RMSE', np.sqrt(crs_mse))
    print("Train MSE", mse)
    print("Train RMSE", np.sqrt(mse))
    y_test_pred = model.predict(test);
    org_sam["Item_Outlet_Sales"] = y_test_pred;
    org_sam.to_csv(fileName, index= False)

In [43]:
# Simple Linear Regression
linn = LinearRegression()
model_test(linn,x_train,y_train,x_test,'simple_linear.csv')

Train MSE 1262822.2312523376
Train RMSE 1123.7536345891556


In [44]:
# Polynomial Linear Regression Degree 2
poly = PolynomialFeatures(degree=2)
x_train_poly = poly.fit_transform(x_train)
x_test_poly = poly.fit_transform(x_test)
model_test(linn,x_train_poly,y_train,x_test_poly,'poly_linear.csv')

Train MSE 969182.5559208856
Train RMSE 984.4706983556624


In [45]:
# Polynomial Linear Regression Degree 3
poly = PolynomialFeatures(degree=3)
x_train_poly = poly.fit_transform(x_train)
x_test_poly = poly.fit_transform(x_test)
model_test(linn,x_train_poly,y_train,x_test_poly,'poly_linear_d3.csv')

Train MSE 276601.6575965128
Train RMSE 525.9293275683652
